In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Disabling this warning as we only need part of the data
# So we copy 'measurements_df' -> 'measurements_subset'
pd.options.mode.chained_assignment = None

In [3]:
measurements_df = pd.read_excel('Data/Elastic Measurements Jul22/Press Machine Sinking Measurements Jul22/Foils experiments 2022_07_18-21.xlsx',sheet_name='Data')

In [4]:
measurements_df.rename(columns={'Unnamed: 1':'parts'},inplace=True)
measurements_df.rename(columns={'item #':'item'},inplace=True)
measurements_df.rename(columns={'forcs (N)':'force (N)'},inplace=True)

measurements_df = measurements_df.drop(measurements_df[measurements_df['item']=='calib1'].index).reset_index(drop=True)
measurements_df.drop(columns=['image','Unnamed: 7'], inplace=True, errors='ignore')

#location_dataset['parts'] = location_dataset['parts'].to_string()
print(measurements_df)

                    item        parts  frame  position (mm) force (N)  \
0    2022-07-18 00:00:00          NaN    NaN            NaN       NaN   
1                      1     Mast TR5    NaN            NaN       NaN   
2                    NaN         bend    1.0          74.34     -0.89   
3                    NaN          NaN    2.0          36.60    -100.1   
4                    NaN          NaN    3.0          -0.10    -199.6   
..                   ...          ...    ...            ...       ...   
476                  NaN          NaN    4.0          67.20    -153.8   
477                  NaN          NaN    NaN            NaN       NaN   
478                  NaN  twist front    1.0          66.52      -0.9   
479                  NaN          NaN    2.0          66.10     -49.8   
480                  NaN          NaN    3.0          65.86    -100.3   

     d_position(mm)  delta y  delta x         M  
0               NaN      NaN      NaN       NaN  
1               NaN    

In [5]:
part_name = None
part_names = []
dates = []
curr_date = curr_part = ''

def not_mast_or_wing(str):
    if str is np.nan:
        return True
    for word in ["Mast","mast","Wing","wing"]:
        if word in str:
            return False
    return True

def is_date(str):
    if type(str) is datetime:
        return True
    return False

In [6]:
for i in range(len(measurements_df)):
    #  Date of measurement
    if(is_date(measurements_df['item'][i])):
        curr_date = measurements_df['item'][i]
    dates.append(curr_date)

    #  Empty 'parts'
    if(measurements_df['parts'][i] is np.nan):
        measurements_df['parts'][i] = curr_part
    else:
        curr_part = measurements_df['parts'][i]

    #  Part names
    if (part_name is None) and (not_mast_or_wing(measurements_df['parts'][i])):
        part_names.append('')
        continue
    if not (not_mast_or_wing(measurements_df['parts'][i])):
        part_name = measurements_df['parts'][i]
        if 'Mast' in part_name:
            part_name = 'm' +part_name[1:]
        if 'Wing' in part_name:
            part_name = 'w' +part_name[1:]
    part_names.append(part_name)

measurements_df['part_name'] = part_names

In [7]:
# We can see in the original table that all slopes are 1 line before the final measurement
# So we'll shift the slope data 1 row beneath
measurements_df['delta y'] = measurements_df['delta y'].shift(periods=1)
measurements_df['delta x'] = measurements_df['delta x'].shift(periods=1)
measurements_df['M'] = measurements_df['M'].shift(periods=1)

In [8]:
measurements_df.drop(columns=['item'], inplace=True, errors='ignore')
measurements_df = measurements_df.dropna(subset=['force (N)']).reset_index(drop=True)

# RECHOOSE FEATURES
# The next subset saves only rows with data at M field (1 row per part-windboard)
measurements_subset = measurements_df.dropna(subset=['M']).reset_index(drop=True)
measurements_subset['item_name'] = measurements_subset['part_name'].str.replace('mast ', '').str.replace('wing ', '')
measurements_subset['item'] = measurements_subset['part_name'].str.slice(0,4)

In [9]:
def calc_scores_factors(forces_df, score_column):
    for index, row in forces_df.iterrows():
        scores = []
        max_score = 4
        curr_df = measurements_subset[(measurements_subset['item'] == row['item']) &
                                      (measurements_subset['parts'] == row['part'])]
        curr_df = curr_df[score_column]
        curr_df = ((curr_df - curr_df.min()) / (curr_df.max() - curr_df.min())) * max_score
        measurements_subset[score_column].iloc[curr_df.index] = curr_df

In [10]:
def calc_score():
    pos_scores = []
    m_scores = []
    forces_df = pd.DataFrame(columns=['item','part','value'])
    forces_df['item'] = ['wing','wing','wing','mast','mast','mast']
    forces_df['part'] = ['bend','twist back', 'twist front','bend','twist back', 'twist front']
    forces_df['value'] = [-300, -100, -100, -250, -150, -100]

    # Normalizing:
    # The reason I haven't chosen a normalization function is because each value
    # Should be multiplied by a different factor
    for index, row in measurements_subset.iterrows():
        #goal_force = forces_df[(forces_df['item'].str.contains(row["part_name"]))]
        goal_force = forces_df[forces_df["item"]==row['item']]
        goal_force = goal_force[goal_force['part']==row['parts']]
        norm_factor = row['force (N)'] / goal_force['value']
        pos_score = row['d_position(mm)'] * norm_factor
        pos_scores.append(pos_score.values[0])
        m_score = row['M'] * norm_factor
        m_scores.append(m_score.values[0])

    measurements_subset['pos_score'] = pos_scores
    measurements_subset['m_score'] = m_scores
    calc_scores_factors(forces_df, 'pos_score')
    calc_scores_factors(forces_df, 'm_score')
    measurements_subset['pos_score'] = measurements_subset['pos_score'] + 1
    measurements_subset['m_score'] = measurements_subset['m_score'] + 1

calc_score()

In [11]:
print(measurements_subset)
%store measurements_subset

          parts  frame  position (mm) force (N)  d_position(mm)  delta y  \
0          bend    4.0         -19.89    -253.6           94.23    19.79   
1    twist back    4.0          74.51    -149.9            3.00     0.91   
2   twist front    3.0          74.34     -96.4            1.82     0.66   
3          bend    4.0         -38.63    -251.5           86.63    17.81   
4    twist back    4.0          43.87    -151.3            2.94     0.91   
..          ...    ...            ...       ...             ...      ...   
94   twist back    4.0          57.99    -150.8            3.78     1.05   
95  twist front    3.0          60.98    -100.6            2.39     1.11   
96         bend    4.0         -14.79    -250.4           85.46    14.64   
97   twist back    4.0          67.20    -153.8            1.79     0.13   
98  twist front    3.0          65.86    -100.3            0.66     0.24   

    delta x         M part_name item_name  item  pos_score   m_score  
0      54.0  0.3